# Encrypted Inference Example

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Datasets
%install '.package(path: "$cwd/ppmlNB_tensor")' ppmlNB_tensor

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Datasets
	.package(path: "/root/swift-ppml/secure_computation/ppmlNB_tensor")
		ppmlNB_tensor
With SwiftPM flags: []
Working in: /tmp/tmpn1ucq_ut/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
import ppmlNB_tensor
import TensorFlow

## Public Training

In [3]:
import Datasets

let epochCount = 12
let batchSize = 128

let dataset = MNIST()

var classifier = Sequential {
    Flatten<Float>()
    Dense<Float>(inputSize: 784, outputSize: 10)
}

let optimizer = SGD(for: classifier, learningRate: 0.1)

print("Beginning training...")

struct Statistics {
    var correctGuessCount: Int = 0
    var totalGuessCount: Int = 0
    var totalLoss: Float = 0
    var batches: Int = 0
}

let testBatches = dataset.testDataset.batched(batchSize)

// The training loop.
for epoch in 1...epochCount {
    var trainStats = Statistics()
    var testStats = Statistics()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(epoch))

    Context.local.learningPhase = .training
    for batch in trainingShuffled.batched(batchSize) {
        let (labels, images) = (batch.label, batch.data)
        // Compute the gradient with respect to the model.
        let 𝛁model = TensorFlow.gradient(at: classifier) { classifier -> Tensor<Float> in
            let ŷ = classifier(images)
            let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
            trainStats.correctGuessCount += Int(
                Tensor<Int32>(correctPredictions).sum().scalarized())
            trainStats.totalGuessCount += batchSize
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.totalLoss += loss.scalarized()
            trainStats.batches += 1
            return loss
        }
        // Update the model's differentiable variables along the gradient vector.
        optimizer.update(&classifier, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in testBatches {
        let (labels, images) = (batch.label, batch.data)
        // Compute loss on test set
        let ŷ = classifier(images)
        let correctPredictions = ŷ.argmax(squeezingAxis: 1) .== labels
        testStats.correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
        testStats.totalGuessCount += batchSize
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.totalLoss += loss.scalarized()
        testStats.batches += 1
    }

    let trainAccuracy = Float(trainStats.correctGuessCount) / Float(trainStats.totalGuessCount)
    let testAccuracy = Float(testStats.correctGuessCount) / Float(testStats.totalGuessCount)
    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(trainStats.totalLoss / Float(trainStats.batches)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(trainAccuracy)), \
        Test Loss: \(testStats.totalLoss / Float(testStats.batches)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(testAccuracy))
        """)
}

Loading resource: train-images-idx3-ubyte
File does not exist locally at expected path: /tmp/MNIST/train-images-idx3-ubyte and must be fetched
Fetching URL: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz...
Writing fetched archive to: /tmp/MNIST/train-images-idx3-ubyte.gz
Archive saved to: /tmp/MNIST/train-images-idx3-ubyte.gz
Extracting archive...
Loading local data at: /tmp/MNIST/train-images-idx3-ubyte
Succesfully loaded resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
File does not exist locally at expected path: /tmp/MNIST/train-labels-idx1-ubyte and must be fetched
Fetching URL: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz...
Writing fetched archive to: /tmp/MNIST/train-labels-idx1-ubyte.gz
Archive saved to: /tmp/MNIST/train-labels-idx1-ubyte.gz
Extracting archive...
Loading local data at: /tmp/MNIST/train-labels-idx1-ubyte
Succesfully loaded resource: train-labels-idx1-ubyte
2019-12-31 10:55:45.725065: W tensorflow/core/fram

In [4]:
let w1 = classifier.layer2.weight
let b1 = classifier.layer2.bias

In [5]:
func get_input() -> (Tensor<Float>, Tensor<Int32>){
    let dataset = MNIST()
    let trainingShuffled = dataset.trainingDataset.shuffled(
        sampleCount: dataset.trainingExampleCount, randomSeed: Int64(1))
    var train_iter = trainingShuffled.self.makeIterator()
    var image = train_iter.next().unsafelyUnwrapped
    return (image.data.reshaped(to: [1, 784]), image.label)
    
}

In [6]:
let (input, label) = get_input()

Loading resource: train-images-idx3-ubyte
Loading local data at: /tmp/MNIST/train-images-idx3-ubyte
Succesfully loaded resource: train-images-idx3-ubyte
Loading resource: train-labels-idx1-ubyte
Loading local data at: /tmp/MNIST/train-labels-idx1-ubyte
Succesfully loaded resource: train-labels-idx1-ubyte
2019-12-31 10:56:23.087928: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of system memory.
Loading resource: t10k-images-idx3-ubyte
Loading local data at: /tmp/MNIST/t10k-images-idx3-ubyte
Succesfully loaded resource: t10k-images-idx3-ubyte
Loading resource: t10k-labels-idx1-ubyte
Loading local data at: /tmp/MNIST/t10k-labels-idx1-ubyte
Succesfully loaded resource: t10k-labels-idx1-ubyte


## Encrypted Inference

In [7]:
let input_private = PrivateTensor.from_values(values: Tensor<Double>(input))
let w1_private = PrivateTensor.from_values(values: Tensor<Double>(w1))
let b1_private = PrivateTensor.from_values(values: Tensor<Double>(b1))

In [8]:
let private_prediction = input_private.matmul(y: w1_private).add(y: b1_private)

In [9]:
let decode_prediction = private_prediction.reveal().decode()